In [1]:
import pandas as pd
import altair as alt
from sublimpy import utils
alt.data_transformers.enable('json')

DataTransformerRegistry.enable('json')

In [2]:
df_nopf = pd.read_parquet("tidy_df_20221101_20230619_noplanar_fit.parquet")
df_pf = pd.read_parquet("tidy_df_20221101_20230619_planar_fit.parquet")
df_pfmp = pd.read_parquet("tidy_df_20221101_20230619_planar_fit_multiplane.parquet")

df_nopf = utils.modify_df_timezone(df_nopf, 'UTC', 'US/Mountain')
df_pf = utils.modify_df_timezone(df_pf, 'UTC', 'US/Mountain')
df_pfmp = utils.modify_df_timezone(df_pfmp, 'UTC', 'US/Mountain')

df = pd.concat([
    df_nopf.query("measurement == 'w'").assign(type = "nopf"),
    df_pf.query("measurement == 'w'").assign(type = "pf"),
    df_pfmp.query("measurement == 'w'").assign(type = "pfmp"),
])

In [3]:
line = alt.Chart().transform_calculate(y = '0').mark_rule().encode(y='y:Q')

plot = alt.Chart().mark_line().encode(
    alt.X('hoursminutes(time):T').axis(labelAlign='center'),
    alt.Y('median(value):Q').title('Vertical wind velocity (m/s)'),
    alt.Color('height:O').scale(scheme='turbo'),
    tooltip='variable',
)

alt.layer(
        line, plot, data = df.query("type == 'pfmp'").query("tower == 'c'")
    ).properties(width = 150, height = 150).facet('tower:N', title='Seemingly local earth coordinates').display(renderer='svg')

alt.FacetChart(...)

In [4]:
(
    alt.layer(
        line, plot, data = df.query("type == 'nopf'")
    ).properties(width = 150, height = 150).facet('tower:N', title='Seemingly local earth coordinates')
    &
    alt.layer(
        line, plot, data = df.query("type == 'pf'")
    ).properties(width = 150, height = 150).facet('tower:N', title='Slope relative local earth coordinates')
    &
    alt.layer(
        line, plot, data = df.query("type == 'pfmp'")
    ).properties(width = 150, height = 150).facet('tower:N', title='Streamwise coordinates')
).display(renderer='svg')

alt.VConcatChart(...)

In [6]:
start_date = '20221101'
end_date = '20230619'

def daily_w_cycle(src, title, ylim):
    return alt.Chart(
        src[src.variable.isin([
            'w_2m_c', 'w_3m_c', 'w_5m_c', 'w_10m_c', 'w_15m_c', 'w_20m_c',
            'w_3m_ue','w_10m_ue',
            'w_3m_uw','w_10m_uw',
            'w_3m_d','w_10m_d',
        ])]
    ).mark_line().encode(
        alt.X('hoursminutes(time):T'),
        alt.Y('median(value):Q').title('Wind speed (m/s)').scale(domain=ylim),
        alt.Color('height:O').scale(scheme='sinebow'),
        alt.StrokeDash('tower:N'),
        tooltip='variable',
    ).properties(width = 200, height = 200, title=title)

(

    daily_w_cycle(
        pd.read_parquet(f"tidy_df_{start_date}_{end_date}_planar_fit.parquet"),
        'SRLEC',
        ylim=[-0.10, 0.05]
    ) | daily_w_cycle(
        pd.read_parquet(f"tidy_df_{start_date}_{end_date}_noplanar_fit.parquet"),
        'Geographic',
        ylim=[-0.10, 0.05]
    ) | 
    daily_w_cycle(
        pd.read_parquet(f"tidy_df_{start_date}_{end_date}_planar_fit_multiplane.parquet"),
        'Streamwise',
        ylim=[-0.10, 0.05]
    )
).properties(title='Vertical wind speed, different coordinate systems').display(renderer='svg')

alt.HConcatChart(...)